In [1]:
import re
import json
import spacy

In [2]:
#STOP WORDS
stop_words = ['in','intersect','except','union','not']
def get_queries(query):
    queries=[]
    runing_query=[]
    i = 1
    runing_query.append(query[0])
    while i < len(query):
        token = query[i]
        if token == '.':
            runing_query.pop()
            i+=1
            continue
        if token == 'as':
            i+=2
            continue 
        if token == 'by' or token == ',' or token == 'distinct': 
            i+=1
            continue
        if token == 'select':
            prev_token = query[i-1]
            nested_query = []
            if prev_token == '(':
                #WHILE TO GET BRACKET END
                open_bracket = 1
                while open_bracket >=1:
                    token = query[i]
                    i+=1
                    if token == '(': open_bracket += 1
                    elif token == ')': open_bracket -= 1
                    if open_bracket == 0: break
                    nested_query.append(token)
            else:
                nested_query = query[i:]
                i = len(query)+1
            nested_queries = get_queries(nested_query)
            queries.extend(nested_queries)
        if token not in stop_words and i <= len(query): runing_query.append(token)
        i+=1
    queries.append(runing_query)
    return queries

In [3]:
bigram_dict = {
    'entity_attributes_select':{},
    'entity_attributes_where':{},
    'entity_attributes_group':{},
    'entity_attributes_having':{},
    'entity_attributes_order':{},
}
trigram_dict = {
    'entity_attributes_select':{},
    'entity_attributes_where':{},
    'entity_attributes_group':{},
    'entity_attributes_having':{},
    'entity_attributes_order':{},
}
dictionaries = {
    'select_attributes':{},
    'entities':{},
    'where_attributes':{},
    'having_attributes':{},
    'group_attributes':{},
    'order_attributes':{}
}
aggregate_attributes = {'max':{},'min':{},'avg':{},'sum':{},'count':{}}
dataSet = []

In [4]:
#Aggregate relations with in select clause
#return arr without max,min,avg,sum,count
def select_aggregate(select_tokens):
    select_attributes = []
    i = 0
    while i < len(select_tokens):
        token = select_tokens[i]
        if token in aggregate_attributes.keys():
            attribute = select_tokens[i+2]
            if aggregate_attributes[token].get(attribute) is None:
                aggregate_attributes[token][attribute] = 0
            aggregate_attributes[token][attribute] += 1
            select_attributes.append(attribute)
            i+=3
        else:
            select_attributes.append(token)
        i+=1

    return select_attributes

In [5]:
#Operations in where clause
#<=,<,>,>=,=,!=,like,not 
#like,in,not in,between,not between
#---------------------------------
#and,or,not
op = ['<=','<','>=','>','=','!=','like','not','in','not in','between','not between','and','or','not','value']
def where_attributes(where_tokens):
    where_attributes = []
    for token in where_tokens:
        if token not in op: where_attributes.append(token)
    return where_attributes


In [6]:
#Language Model
def generate_bigram(entities,attributes,dict_type):
    for entity in entities:
        for attribute in attributes:
            bigram = entity + ' ' + attribute
            if bigram_dict[dict_type].get(bigram) is None:
                bigram_dict[dict_type][bigram] = 0
            bigram_dict[dict_type][bigram] += 1

#Language Model
def generate_trigram(entities,attributes,dict_type):
    entities.sort()
    for i in range(len(entities)):
        for j in range(i+1,len(entities)):
            for attribute in attributes:
                trigram = entities[i] + ' ' + entities[j] + ' ' + attribute
                if trigram_dict[dict_type].get(trigram) is None:
                    trigram_dict[dict_type][trigram] = 0
                trigram_dict[dict_type][trigram] += 1

In [7]:
nlp = spacy.load('en_core_web_sm')

def get_lemma(text):
    doc = nlp(text)
    return doc[0].lemma_

def camel_case_paskal_split(identifier):
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    return [m.group(0) for m in matches]

def cleanName(colName):
    #remove names with one character 
    if len(colName) <= 1: return colName
    #seperate pascal and camel cases
    Names = camel_case_paskal_split(colName)
    clean_Names=[]
    for name in Names:
      #remove names that are only numbers
      result = re.search("^[ 0-9]+$", name)
      if result is not None: continue
      #remove numbers at the end of the string 
      name = re.sub("[0-9]+$", '', name)
      #lower
      name = name.lower()
      lemma = get_lemma(name)
      clean_Names.append(lemma)
      
    return clean_Names
    
def cleanEntityName(entityName):
  entityName = entityName.strip()
  entityName = re.split('_| |-',entityName)
  entity_clean=[]
  for name in entityName:
    clean_name = cleanName(name)
    entity_clean.extend(clean_name)
  return entity_clean


In [10]:
def generate_unigram(query):
    query_dicts = {'select_attributes':[],'entities':[],
    'where_attributes':[],
    'having_attributes':[],
    'group_attributes':[],
    'order_attributes':[],
    "select_attributes_clean":[],
    "where_attributes_clean":[],
    }
    current_dict = 'select_attributes'
    for token in query:
        if token =='select':
            continue

        if token =='from' or token == 'join':
            current_dict = 'entities'
            continue

        if token =='where' or token == 'on':
            current_dict = 'where_attributes'
            continue

        if token =='group':
            current_dict = 'group_attributes'
            continue

        if token == 'having':
            current_dict = 'having_attributes'
            continue

        if token == 'order':
            current_dict = 'order_attributes'
            continue
        
        if token == 'limit': return query_dicts


        if current_dict == 'entities':
            token = cleanEntityName(token)
            for t in token:
                if dictionaries[current_dict].get(t) is None:
                    dictionaries[current_dict][t] = 0
                dictionaries[current_dict][t] += 1
                query_dicts[current_dict].append(t)
            continue

        if dictionaries[current_dict].get(token) is None:
            dictionaries[current_dict][token] = 0

        dictionaries[current_dict][token] += 1
        query_dicts[current_dict].append(token)
    return query_dicts

In [11]:
def get_vocab(dataSet):
    vocab = set()
    for queriy in dataSet:
        for entity in queriy['entities']:
            vocab.add(entity)
    return vocab

In [12]:
queries = []
with open('/home/hager/college/GP/NLP2SQL/Search Engine/train_spider.json', 'r',encoding='UTF-8') as file:
    data = json.load(file)
    for query in data:
        query_tokens = query['query_toks_no_value']
        current_queries = get_queries(query_tokens)      
        queries.extend(current_queries)

In [13]:
for query in queries:
    q_data = generate_unigram(query)
    q_data['select_attributes_clean'] = select_aggregate(q_data['select_attributes'])
    q_data['where_attributes_clean'] = where_attributes(q_data['where_attributes'])
    generate_bigram(q_data['entities'],q_data['select_attributes_clean'],'entity_attributes_select')
    generate_bigram(q_data['entities'],q_data['where_attributes_clean'],'entity_attributes_where')
    '''
    generate_trigram(q_data['entities'],q_data['select_attributes_clean'],'entity_attributes_select')
    generate_trigram(q_data['entities'],q_data['where_attributes_clean'],'entity_attributes_where')
    '''
    dataSet.append(q_data)

In [14]:
print(len(dataSet))
print(dataSet[19])

8062
{'select_attributes': ['count', '(', '*', ')'], 'entities': ['farm'], 'where_attributes': [], 'having_attributes': [], 'group_attributes': [], 'order_attributes': [], 'select_attributes_clean': ['*'], 'where_attributes_clean': []}


In [98]:
vocab = get_vocab(dataSet)
print(len(vocab))
print(vocab)

423
{'unit', 'flight', 'safety', 'service', 'in', 'premise', 'availability', 'physician', 'build', 'stock', 'music', 'head', 'additional', 'wine', 'sale', 'staff', 'aircraft', 'market', 'amenity', 'repair', 'live', 'activity', 'supplier', 'fame', 'mediatype', 'teach', 'claim', 'phone', 'outcome', 'user', 'artwork', 'membership', 'player', 'useracct', 'regular', 'student', 'delivery', 'individual', 'performer', 'circuit', 'band', 'drama', 'tweet', 'characteristic', 'policy', 'customer', 'discount', 'tourist', 'with', 'vote', 'view', 'league', 'actual', 'copy', 'county', 'instrument', 'route', 'injury', 'processing', 'exhibition', 'registration', 'elimination', 'workshop', 'match', 'management', 'camera', 'calendar', 'musical', 'roller', 'mailshot', 'contract', 'course', 'to', 'apartment', 'nomination', 'personfriend', 'invoice', 'chain', 'trip', 'branch', 'first', 'truck', 'client', 'financial', 'grape', 'journalist', 'register', 'round', 'review', 'architect', 'agent', 'builder', 'avai

In [102]:
import pickle
with open('/home/hager/college/GP/NLP2SQL/Search Engine/dataSet.pickle', 'wb') as handle:
    pickle.dump(dataSet, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('/home/hager/college/GP/NLP2SQL/Search Engine/vocab.pickle', 'wb') as handle:
    pickle.dump(vocab, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [104]:
with open('/home/hager/college/GP/NLP2SQL/Search Engine/dataSet.pickle', 'rb') as handle:
    dataSetL = pickle.load(handle)
print(len(dataSetL))

10337


In [4]:
testSchema = {
    11: 
    {'TableName': 'DEPARTMENT', 
    'TableType':'',
    'attributes': {'name': 'str', 
    'start_date': 'datetime',
    'EMPLOYEE_Manages': 'str'}, 
    'primaryKey': ['name'], 
    'ForgeinKey': [{'attributeName': 'EMPLOYEE_Manages',
    'ForignKeyTable': 'EMPLOYEE', 
    'ForignKeyTableAttributeName': 'ssn', 
    'patricipaction': 'partial', 
    'dataType': 'str'}], 
    'isWeak': False},
    34: 
    {'TableName': 'DEPARTMENT_Clocation', 
    'TableType':'',
    'attributes': {'Clocation': 'str',
    'DEPARTMENT_name': 'str'}, 
    'primaryKey': ['Clocation', 
    'DEPARTMENT_name'], 
    'ForgeinKey': [{'attributeName': 'DEPARTMENT_name', 
    'ForignKeyTable': 'DEPARTMENT', 
    'ForignKeyTableAttributeName': 'name', 
    'patricipaction': 'full', 
    'dataType': 'str'}], 
    'isWeak': False}, 
    12: 
    {'TableName': 'EMPLOYEE',
    'TableType':'',
    'attributes': {'last_name': 'str', 
    'middle_initis': 'str', 
    'first_name': 'str', 
    'address': 'str',
    'salary': 'float',
    'sex': 'str', 
    'status': 'str', 
    'birth_dat': 'str', 
    'ssn': 'str',
    'start_date': 'datetime',
    'DEPARTMENT_Employed_name': 'str',
    'EMPLOYEE_Supervision_': 'str'},
    'primaryKey': ['ssn'], 
    'ForgeinKey': [{'attributeName': 'DEPARTMENT_Employed_name',
    'ForignKeyTable': 'DEPARTMENT', 'ForignKeyTableAttributeName': 'name',
    'patricipaction': 'full', 'dataType': 'str'}, 
    {'attributeName': 'EMPLOYEE_Supervision_', 
    'ForignKeyTable': 'EMPLOYEE', 
    'ForignKeyTableAttributeName': 'ssn',
    'patricipaction': 'partial', 
    'dataType': 'str'}], 
    'isWeak': False},
    24: {'TableName': 'PROJECT', 
    'TableType':'',
    'attributes': {'location': 'str',
    'name': 'str', 
    'budget': 'float',
    'DEPARTMENT_Assigned_name': 'str'}, 
    'primaryKey': ['name'], 
    'ForgeinKey': [{'attributeName': 'DEPARTMENT_Assigned_name',
    'ForignKeyTable': 'DEPARTMENT', 
    'ForignKeyTableAttributeName': 'name',
    'patricipaction': 'partial', 
    'dataType': 'str'}], 
    'isWeak': False}, 
    25: 
    {'TableName': 'DEPENDENT',
    'TableType':'',
    'attributes': {'sex': 'str', 
    'relatlonship': 'str',
    'name': 'str',
    'birth_date': 'datetime', 
    'Dependents_EMPLOYEE_': 'str'}, 
    'primaryKey': ['Dependents_EMPLOYEE_'], 
    'ForgeinKey': [{'attributeName': 'Dependents_EMPLOYEE_', 
    'ForignKeyTable': 'EMPLOYEE', 
    'ForignKeyTableAttributeName': 'ssn', 
    'patricipaction': 'partial', 
    'dataType': 'str'}], 
    'isWeak': True}, 
    35: 
    {'TableName': 'Works_EMPLOYEE_PROJECT', 
    'TableType':'mTm',
    'attributes': {'start_date': 'datetime', 
    'hours': 'int', 
    'EMPLOYEE_': 'str', 
    'PROJECT_': 'str'}, 
    'primaryKey': ['EMPLOYEE_', 'PROJECT_'], 
    'ForgeinKey': [{'attributeName': 'EMPLOYEE_', 
    'ForignKeyTable': 'EMPLOYEE',
    'ForignKeyTableAttributeName': 'ssn', 
    'patricipaction': 'full',
    'dataType': 'str'}, 
    {'attributeName': 'PROJECT_',
    'ForignKeyTable': 'PROJECT', 
    'ForignKeyTableAttributeName': 'name',
    'patricipaction': 'full',
    'dataType': 'str'}], 
    'isWeak': False}}

import pickle
with open('/home/hager/college/GP/GP/src/SearchEngine/testSchema.pickle', 'wb') as handle:
    pickle.dump(testSchema, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
def bestJoins(paths):
    minJoins = float("inf")
    Join = None
    for path in paths:
        joinsCount = path.count(' AND ')
        if joinsCount < minJoins:
            minJoins = joinsCount
            Join = path
    
    filteredJoins = []
    prev_joins = set()
    for path in paths:
        if path.count(' AND ') == minJoins:
            joins =  "".join(sorted(re.split(" AND | = ",path.strip())))
            if joins in prev_joins: continue
            filteredJoins.append(path)
            prev_joins.add(joins)
    return filteredJoins

#TODO:give to this function only a connected component graph
def findPathsBFS(source,goals,graph):
    queue = deque()
    queue.append((source,""))
    paths = set()
    visited = set()
    visited.add(source)
    while queue:
        curEntity,path = queue.popleft()
        curEntityName = curEntity
        for neighbor in graph[curEntityName]:
            neighborJoin = neighbor['ForignKeyTable']+'.'+neighbor['ForignKeyTableAttributeName']
            newPath = ' '+curEntityName +'.'+neighbor["attributeName"]+ ' = ' +neighborJoin
            if path != '': newPath = path+' AND'+ newPath

            print("path:",newPath , "---- matching:",[" "+entity+"." in newPath for entity in goals])
            ''
            if neighbor['ForignKeyTable'] in goals and neighborJoin not in path\
                 and all(" "+entity+"." in newPath for entity in goals):
                    # print(neighbor['ForignKeyTable'] , newPath)
                    paths.add(newPath)
            #print("visited",neighbor['ForignKeyTable'] not in visited,visited,neighbor['ForignKeyTable'])
            if neighbor['ForignKeyTable'] not in visited:
                print(neighbor['ForignKeyTable'])
                visited.add(neighbor['ForignKeyTable'])
                queue.append((neighbor['ForignKeyTable'],newPath))
    #print("visited------------------------")
    #print(visited)
    return paths


In [197]:
trigram_dict['entity_attributes_select']

{'department head creation': 1,
 'department management creation': 1,
 'head management creation': 1,
 'department management name': 2,
 'department management num_employees': 1,
 'head management age': 1,
 'department head born_state': 2,
 'department management born_state': 2,
 'head management born_state': 2,
 'department management department_id': 1,
 'department management *': 1,
 'city farm_competition year': 2,
 'city farm_competition official_name': 4,
 'city farm_competition status': 2,
 'city farm_competition theme': 2,
 'courses student_course_registrations course_name': 5,
 'candidates people first_name': 2,
 'candidates people last_name': 2,
 'courses student_course_registrations student_id': 3,
 'student_course_registrations students student_details': 4,
 'courses student_course_attendance *': 4,
 'candidates people candidate_id': 2,
 'student_course_registrations students student_id': 2,
 'student_course_registrations students *': 4,
 'courses student_course_registration

In [3]:

if all(entity in join for entity in mapped_entities):
    print('True')

False


In [7]:
from nltk.corpus import wordnet
syns = wordnet.synsets("instructor")
print(syns)

[Synset('teacher.n.01')]


In [185]:
#TODO
#Merge with aggreagtion of select attributes???
#handle having aggregations
#handle order by ASC/DESC
#handle duplicate attributes in where clause
#attribute from same entity in select and where clause

##################################################################


#Remove After 'as'
#stop wored ''
#check prev
#From,Join ===> Entity Name
#On,Where ===> Attribute Name 
#or token == 'group' or token == 'limit' or token == 'offset' or token == 'having'
#Aggregation in select attributes

# entity => e ,attributes => a
# SELECT TO FROM ===> Attribute Name
# From TO WHERE ==> Entity Name (REMOVE JOIN,AS,After as|| ON attribte=> Split on '.')
# NESTED select do recursion with brackets(())
# SELECT after Intersect
# SELECT after Union
# SELECT after Except
# IN_NOT_IN
###############################
# WHERE Type of Condition
# Group By
# Order By
# Limit
# Offset
# Having

In [ ]:
#TODO
#tokenize dataset like spider 
#lemmatize
#remove '_'  ===> as general as possible
#embeddings
#distance between words